In [1]:
import pandas as pd
import numpy as np
import math

In [3]:
Ratings=pd.read_csv("C:/Users/tomar/OneDrive/Desktop/Recommender system/ratings.csv",encoding="ISO-8859-1")
Movies=pd.read_csv("C:/Users/tomar/OneDrive/Desktop/Recommender system/movies.csv",encoding="ISO-8859-1")
Tags=pd.read_csv("C:/Users/tomar/OneDrive/Desktop/Recommender system/tags.csv",encoding="ISO-8859-1")

In [18]:
a=Ratings[Ratings['userId']==320]
a[a['movieId']==260]

,userId,movieId,rating,timestamp
108,320,260,4.5,1145964264


In [4]:
TF= Tags.groupby(['movieId','tag'], as_index = False, sort = False).count().rename(columns = {'userId': 'tag_count_TF'})[['movieId','tag','tag_count_TF']]

In [5]:
Tag_distinct = Tags[['tag','movieId']].drop_duplicates()
DF =Tag_distinct.groupby(['tag'], as_index = False, sort = False).count().rename(columns = {'movieId': 'tag_count_DF'})[['tag','tag_count_DF']]
a=math.log10(len(np.unique(Tags['movieId'])))
DF['IDF']=a-np.log10(DF['tag_count_DF'])
TF = pd.merge(TF,DF,on = 'tag', how = 'left', sort = False)
TF['TF-IDF']=TF['tag_count_TF']*TF['IDF']

In [6]:
Vect_len=TF[['movieId','TF-IDF']]
Vect_len['TF-IDF-Sq']=Vect_len['TF-IDF']**2
Vect_len =Vect_len.groupby(['movieId'], as_index = False, sort = False).sum().rename(columns = {'TF-IDF-Sq': 'TF-IDF-Sq-sum'})[['movieId','TF-IDF-Sq-sum']]
Vect_len['vect_len'] = np.sqrt(Vect_len[['TF-IDF-Sq-sum']].sum(axis=1))
TF = pd.merge(TF,Vect_len,on = 'movieId', how = 'left', sort = False)
TF['TAG_WT']=TF['TF-IDF']/TF['vect_len']

C:\Users\tomar\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [20]:
Ratings_filter=Ratings[Ratings['rating']>=3.5]
distinct_users=np.unique(Ratings['userId'])
user_tag_pref=pd.DataFrame()
i=1

#enter user ID for analysis
userID = 320

user_index = distinct_users.tolist().index(userID)

for user in distinct_users[user_index:user_index+1]:
    
    if i%30==0:
        print ("user: ", i , "out of: ", len(distinct_users))
            
    user_data=  Ratings_filter[Ratings_filter['userId']==user]
    user_data = pd.merge(TF,user_data,on = 'movieId', how = 'inner', sort = False)
    user_data1 = user_data.groupby(['tag'], as_index = False, sort = False).sum().rename(columns = {'TAG_WT': 'tag_pref'})[['tag','tag_pref']]
    user_data1['user']=user
    user_tag_pref = user_tag_pref.append(user_data1, ignore_index=True)
    i=i+1

In [24]:
distinct_users=np.unique(Ratings_filter['userId'])
tag_merge_all=pd.DataFrame()

i=1

#enter user ID for analysis
userID = 320

user_index = distinct_users.tolist().index(userID)

for user in distinct_users[user_index:user_index+1]:

    
    user_tag_pref_all=  user_tag_pref[user_tag_pref['user']==user]
    distinct_movies = np.unique(TF['movieId'])
    j=1
    for movie in distinct_movies:
        
        if j%300==0:
            
            print ("movie: ", j , "out of: ", len(distinct_movies) , "with user: ", i , "out of: ", len(distinct_users))
        
        TF_Movie=  TF[TF['movieId']==movie]
        tag_merge = pd.merge(TF_Movie,user_tag_pref_all,on = 'tag', how = 'left', sort = False)
        tag_merge['tag_pref']=tag_merge['tag_pref'].fillna(0)
        tag_merge['tag_value']=tag_merge['TAG_WT']*tag_merge['tag_pref']
        
        TAG_WT_val=np.sqrt(np.sum(np.square(tag_merge['TAG_WT']), axis=0))
        tag_pref_val=np.sqrt(np.sum(np.square(user_tag_pref_all['tag_pref']), axis=0))
        
        tag_merge_final = tag_merge.groupby(['user','movieId'])[['tag_value']].sum().rename(columns = {'tag_value': 'Rating'}).reset_index()
        
        tag_merge_final['Rating']=tag_merge_final['Rating']/(TAG_WT_val*tag_pref_val)
        
        tag_merge_all = tag_merge_all.append(tag_merge_final, ignore_index=True)
        j=j+1
    i=i+1
tag_merge_all = tag_merge_all.sort_values(['user','Rating'], ascending=False)

# Removing movies already rated by user

movies_rated = Ratings[Ratings['userId']==userID]['movieId']
tag_merge_all = tag_merge_all[~tag_merge_all['movieId'].isin(movies_rated)]

movie:  300 out of:  2495 with user:  1 out of:  862
movie:  600 out of:  2495 with user:  1 out of:  862
movie:  900 out of:  2495 with user:  1 out of:  862
movie:  1200 out of:  2495 with user:  1 out of:  862
movie:  1500 out of:  2495 with user:  1 out of:  862
movie:  1800 out of:  2495 with user:  1 out of:  862
movie:  2100 out of:  2495 with user:  1 out of:  862
movie:  2400 out of:  2495 with user:  1 out of:  862


In [25]:
tag_merge_all

,user,movieId,Rating
29,320.0,32.0,0.349188
814,320.0,1748.0,0.306611
560,320.0,1206.0,0.297872
2200,320.0,48394.0,0.288340
554,320.0,1199.0,0.287614
2116,320.0,32587.0,0.287348
621,320.0,1270.0,0.283335
503,320.0,1089.0,0.278155
564,320.0,1210.0,0.276866
1987,320.0,7147.0,0.264006


In [63]:
distinct_users=np.unique(Ratings['userId'])
user_tag_pref=pd.DataFrame()
i=1

#enter user ID for analysis
userID = 320

user_index = distinct_users.tolist().index(userID)

for user in distinct_users[user_index:user_index+1]:    
    if i%30==0:
        print ("user: ", i , "out of: ", len(distinct_users))
            
    user_data=  Ratings[Ratings['userId']==user]
    user_data['weight']=user_data["rating"]-user_data["rating"].mean()
    user_data1 = pd.merge(TF,user_data,on = 'movieId', how = 'inner', sort = False)
    user_data1['TAG_WT_WTD'] = user_data1['TAG_WT']*user_data1['weight']
    user_data2 = user_data1.groupby(['tag'], as_index = False, sort = False).sum().rename(columns = {'TAG_WT_WTD': 'tag_pref'})[['tag','tag_pref']]
    user_data2['user']=user
    user_tag_pref = user_tag_pref.append(user_data2, ignore_index=True)
    i=i+1

C:\Users\tomar\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [64]:
distinct_users=np.unique(Ratings_filter['userId'])
tag_merge_all=pd.DataFrame()

i=1

#enter user ID for analysis
userID = 320

user_index = distinct_users.tolist().index(userID)

for user in distinct_users[user_index:user_index+1]:

    
    user_tag_pref_all=  user_tag_pref[user_tag_pref['user']==user]
    distinct_movies = np.unique(TF['movieId'])
    j=1
    for movie in distinct_movies:
        
        if j%300==0:
            
            print ("movie: ", j , "out of: ", len(distinct_movies) , "with user: ", i , "out of: ", len(distinct_users))
        
        TF_Movie=  TF[TF['movieId']==movie]
        tag_merge = pd.merge(TF_Movie,user_tag_pref_all,on = 'tag', how = 'left', sort = False)
        tag_merge['tag_pref']=tag_merge['tag_pref'].fillna(0)
        tag_merge['tag_value']=tag_merge['TAG_WT']*tag_merge['tag_pref']
        
        TAG_WT_val=np.sqrt(np.sum(np.square(tag_merge['TAG_WT']), axis=0))
        tag_pref_val=np.sqrt(np.sum(np.square(user_tag_pref_all['tag_pref']), axis=0))
        
        tag_merge_final = tag_merge.groupby(['user','movieId'])[['tag_value']].sum().rename(columns = {'tag_value': 'Rating'}).reset_index()
        
        tag_merge_final['Rating']=tag_merge_final['Rating']/(TAG_WT_val*tag_pref_val)
        
        tag_merge_all = tag_merge_all.append(tag_merge_final, ignore_index=True)
        j=j+1
    i=i+1
tag_merge_all = tag_merge_all.sort_values(['user','Rating'], ascending=False)

# Removing movies already rated by user

movies_rated = Ratings[Ratings['userId']==userID]['movieId']
tag_merge_all = tag_merge_all[~tag_merge_all['movieId'].isin(movies_rated)]

movie:  300 out of:  2495 with user:  1 out of:  862
movie:  600 out of:  2495 with user:  1 out of:  862
movie:  900 out of:  2495 with user:  1 out of:  862
movie:  1200 out of:  2495 with user:  1 out of:  862
movie:  1500 out of:  2495 with user:  1 out of:  862
movie:  1800 out of:  2495 with user:  1 out of:  862
movie:  2100 out of:  2495 with user:  1 out of:  862
movie:  2400 out of:  2495 with user:  1 out of:  862


In [65]:
tag_merge_all

,user,movieId,Rating
2216,320.0,48394.0,0.203681
2006,320.0,7153.0,0.184239
571,320.0,1210.0,0.178088
337,320.0,628.0,0.177350
1316,320.0,2997.0,0.175898
510,320.0,1089.0,0.165156
2003,320.0,7147.0,0.161338
2132,320.0,32587.0,0.160441
2220,320.0,48780.0,0.159790
561,320.0,1199.0,0.154699
